In [1]:
import keras
import numpy as np
import tensorflow as tf
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

In [4]:
xy = np.loadtxt('ThoraricSurgery.csv', delimiter=',', dtype=np.float32)

In [5]:
xy = MinMaxScaler(xy)
print(xy)

[[0.6226013  0.         0.4855967  ... 0.         0.6212121  0.        ]
 [0.         0.14285715 0.2962963  ... 0.         0.59090906 0.        ]
 [0.01492537 0.14285715 0.3600823  ... 0.         0.6818182  0.9999999 ]
 ...
 [0.86353946 0.71428573 0.80658436 ... 0.         0.6212121  0.        ]
 [0.05117271 1.         0.5925926  ... 0.         0.56060606 0.9999999 ]
 [0.9509595  1.         0.7736625  ... 0.         0.42424244 0.        ]]


In [6]:
x_train = xy[:400, 0:-1]
y_train = xy[:400, [-1]]

x_test = xy[400:, 0:-1]
y_test = xy[400:, [-1]]

print(x_train.shape, x_test.shape) 

(400, 17) (70, 17)


In [7]:
from keras import models
from keras import layers
from keras import regularizers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(30,kernel_regularizer=regularizers.l2(0.001),
                          activation='relu',input_shape=(17,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# k-겹 교차 검증
# x_train, y_train을 다시 train data(partial) 300개, validation data 100개로 나눈다.
k = 4
num_val_samples = len(x_train) // k
print(num_val_samples)         # validation data: 100개
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비
    partial_train_data = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성
    model = build_model()
    # 모델 훈련
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

100
처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [9]:
all_scores

[0.800000011920929, 0.8500000238418579, 0.8500000238418579, 0.9200000166893005]

In [10]:
np.mean(all_scores)

0.8550000190734863

In [11]:
# 최종 모델 훈련
model = build_model()
model.fit(x_train, y_train, epochs=100, batch_size=10)
results=model.evaluate(x_test, y_test)

Epoch 1/100
400/400 [==============================] - 0s 539us/step - loss: 0.2775 - accuracy: 0.4500
Epoch 2/100
400/400 [==============================] - 0s 122us/step - loss: 0.2006 - accuracy: 0.7650
Epoch 3/100
400/400 [==============================] - 0s 122us/step - loss: 0.1646 - accuracy: 0.8400
Epoch 4/100
400/400 [==============================] - 0s 127us/step - loss: 0.1547 - accuracy: 0.8525
Epoch 5/100
400/400 [==============================] - 0s 130us/step - loss: 0.1511 - accuracy: 0.8625
Epoch 6/100
400/400 [==============================] - 0s 127us/step - loss: 0.1496 - accuracy: 0.8600
Epoch 7/100
400/400 [==============================] - 0s 130us/step - loss: 0.1397 - accuracy: 0.8600
Epoch 8/100
400/400 [==============================] - 0s 145us/step - loss: 0.1377 - accuracy: 0.8625
Epoch 9/100
400/400 [==============================] - 0s 182us/step - loss: 0.1469 - accuracy: 0.8600
Epoch 10/100
400/400 [==============================] - 0s 150us/step - l

In [12]:
results

[0.17457109221390316, 0.7857142686843872]